In [49]:
import json
import pandas as pd

In [2]:
with open('Data/vignettes.json','r') as f:
    vignettes = json.load(f)

In [39]:
cases = list(vignettes.values())

In [44]:
print(len(cases))

1671


In [206]:
cases[12]

{'card': {'diseases': [{'id': '719a1708-feb5-483a-91f8-c772c8efc1e8',
    'rareness': 'very_rare'}],
  'duration': 'C0436363',
  'id': 448,
  'symptoms': [{'concept': {'id': 'b55a0003-ccf8-415c-b212-40f6ab4ec0b3'},
    'label': 'Symptom',
    'severity': 'PRESENT'},
   {'concept': {'id': '923c9bf2-68d8-4122-8367-8af5f1653a59'},
    'label': 'Symptom',
    'severity': 'PRESENT'},
   {'concept': {'id': '4556138c-1ac8-4b4c-aef8-4353aa556932'},
    'label': 'Symptom',
    'severity': 'PRESENT'},
   {'concept': {'id': 'f5aaa5a5-b6b7-430d-b9e5-730794a794f1'},
    'label': 'Symptom',
    'severity': 'PRESENT'},
   {'concept': {'id': '35e2aeb9-932a-40b4-85fa-80c4b3ce21e2'},
    'label': 'Symptom',
    'severity': 'PRESENT'},
   {'concept': {'id': '2234f4b8-fb8e-4d11-88f5-6054cb34db12'},
    'label': 'Symptom',
    'severity': 'PRESENT'}],
  'risk_factors': [{'concept': {'id': '74189f81-1a5d-481c-b2e3-7bdf42fd7814',
     'in_model': True},
    'label': 'Risk',
    'presence': 'PRESENT'}],
  'ne

In [41]:
ids = [x['card']['id'] for x in cases]

Below code shows that each case only has one disease (good - makes it easier)

In [43]:
for x in cases:
    if len(x['card']['diseases']) > 1:
        print(x['card']['id'])

In [45]:
diseases = [x['card']['diseases'][0]['id'] for x in cases]

In [46]:
rareness_of_diseases = [x['card']['diseases'][0]['rareness'] for x in cases]

In [47]:
duration = [x['card']['duration'] for x in cases]

In [48]:
network_name = [x['card']['network_name'] for x in cases]

In [51]:
disease_df = pd.DataFrame({'ids': ids, 'diseases': diseases, 'rareness_of_diseases': rareness_of_diseases, 'duration': duration, 'network_name': network_name})
disease_df.set_index('ids',inplace=True)

In [52]:
disease_df.head()

,diseases,rareness_of_diseases,duration,network_name
ids,,,,
427,6ad6f29a-a992-41b9-9950-9d2a19f1c5ed,common,C0436365,G
428,98c37437-27fa-4ba2-a84b-b10df6ab3176,uncommon,C0436363,A
429,c1bb7409-baeb-481d-a06f-88f82dd01838,very_common,C0436363,H
430,88bc30dd-cedf-48c5-9dc5-52961d641c39,common,C0436363,A
431,5de08d77-ee18-4655-9d90-d56a160601fa,very_rare,C0436365,C


In [56]:
disease_df.groupby('diseases').size().describe()

count    387.000000
mean       4.317829
std        2.532640
min        1.000000
25%        2.000000
50%        4.000000
75%        6.000000
max       15.000000
dtype: float64

In [67]:
disease_count_series = disease_df.groupby('diseases').size()
disease_count_series[disease_count_series == 1].sum()

50

In [65]:
disease_df.groupby('rareness_of_diseases').size().sort_values()

rareness_of_diseases
almost_impossible     18
very_common          131
very_rare            210
rare                 353
common               413
uncommon             546
dtype: int64

In [64]:
disease_df.groupby('duration').size()

duration
C0436361     37
C0436362    201
C0436363    583
C0436364    339
C0436365    511
dtype: int64

In [68]:
disease_df.groupby('network_name').size()

network_name
A    199
B    275
C    224
D    108
E     20
F    200
G    329
H    212
I     89
J     15
dtype: int64

In [225]:
symptoms = []
for x in cases:
    for y in x['card']['symptoms']:
        symptom_id = y['concept']['id']
        if symptom_id not in symptoms:
            symptoms.append(symptom_id)
    

In [226]:
len(symptoms)

552

That is a lot of symptoms here... may cause issues

In [72]:
symptom_case_dict = {}

for symptom_id in symptoms:
    symptom_case_dict[symptom_id] = []


In [75]:
for case in cases:
    case_symptoms = case['card']['symptoms']
    case_symptoms_ids = [x['concept']['id'] for x in case_symptoms]
    
    for symptom_id in symptoms:
        if symptom_id in case_symptoms_ids:
            idx = case_symptoms_ids.index(symptom_id)
            severity = case_symptoms[idx]['severity']
            symptom_case_dict[symptom_id].append(severity)
        else:
            symptom_case_dict[symptom_id].append(None)
            
        


In [76]:
symptom_df = pd.DataFrame(symptom_case_dict, index = ids)

In [77]:
symptom_df.head()

,e97c3c8f-fb0d-4817-b6fc-a4a22db6a581,2234f4b8-fb8e-4d11-88f5-6054cb34db12,a40ffca1-8c4d-4a3a-9eaa-8536f4230a43,3948e1d7-a894-4dc9-b293-45be82543289,0e7faf20-6073-49ce-a037-1b7eeb443117,cc0446f9-77b4-41bb-9aa1-c0976a5dcb06,35e2aeb9-932a-40b4-85fa-80c4b3ce21e2,923c9bf2-68d8-4122-8367-8af5f1653a59,172d1f85-8f42-444d-b407-5392fa924d94,b8d4f595-ac87-4f1d-873a-030b803737b8,...,f5aebcd8-2bef-469c-918a-4d92db20b3f7,11dc2a44-fc16-4ed0-9675-458e7ee4fcfb,6723e63a-92af-45fa-8437-42972c4d6443,cd6f9a42-3fe5-4ecc-9dcb-f16551b260e7,9dd7de35-2d03-47c6-aa77-fab6c13ddc23,86b50a9e-2f01-4114-abfc-c184609fe71f,d4342049-f405-4e22-a09e-a034c95bfd8a,4673974b-9d1b-43fd-b61f-4b1044c67ace,647dd3d9-724f-4a98-b84d-e6a2105fa5f0,1f394df4-70cb-4226-8123-2762abb7ba4f
427,MODERATE,MILD,MILD,MILD,MILD,NOT_PRESENT,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
428,None,None,None,None,None,None,PRESENT,PRESENT,PRESENT,PRESENT,...,None,None,None,None,None,None,None,None,None,None
429,None,PRESENT,PRESENT,None,PRESENT,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
430,None,MILD,None,NOT_PRESENT,None,None,None,PRESENT,MODERATE,None,...,None,None,None,None,None,None,None,None,None,None
431,None,NOT_PRESENT,None,NOT_PRESENT,None,NOT_PRESENT,None,None,NOT_PRESENT,None,...,None,None,None,None,None,None,None,None,None,None


In [78]:
symptom_df = symptom_df.rename_axis("symptom_ids", axis = 'columns')

In [84]:
symptom_df

symptom_ids,e97c3c8f-fb0d-4817-b6fc-a4a22db6a581,2234f4b8-fb8e-4d11-88f5-6054cb34db12,a40ffca1-8c4d-4a3a-9eaa-8536f4230a43,3948e1d7-a894-4dc9-b293-45be82543289,0e7faf20-6073-49ce-a037-1b7eeb443117,cc0446f9-77b4-41bb-9aa1-c0976a5dcb06,35e2aeb9-932a-40b4-85fa-80c4b3ce21e2,923c9bf2-68d8-4122-8367-8af5f1653a59,172d1f85-8f42-444d-b407-5392fa924d94,b8d4f595-ac87-4f1d-873a-030b803737b8,...,f5aebcd8-2bef-469c-918a-4d92db20b3f7,11dc2a44-fc16-4ed0-9675-458e7ee4fcfb,6723e63a-92af-45fa-8437-42972c4d6443,cd6f9a42-3fe5-4ecc-9dcb-f16551b260e7,9dd7de35-2d03-47c6-aa77-fab6c13ddc23,86b50a9e-2f01-4114-abfc-c184609fe71f,d4342049-f405-4e22-a09e-a034c95bfd8a,4673974b-9d1b-43fd-b61f-4b1044c67ace,647dd3d9-724f-4a98-b84d-e6a2105fa5f0,1f394df4-70cb-4226-8123-2762abb7ba4f
427,MODERATE,MILD,MILD,MILD,MILD,NOT_PRESENT,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
428,None,None,None,None,None,None,PRESENT,PRESENT,PRESENT,PRESENT,...,None,None,None,None,None,None,None,None,None,None
429,None,PRESENT,PRESENT,None,PRESENT,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
430,None,MILD,None,NOT_PRESENT,None,None,None,PRESENT,MODERATE,None,...,None,None,None,None,None,None,None,None,None,None
431,None,NOT_PRESENT,None,NOT_PRESENT,None,NOT_PRESENT,None,None,NOT_PRESENT,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7772,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7784,None,None,None,None,None,None,None,PRESENT,NOT_PRESENT,None,...,None,None,None,None,None,None,None,None,None,None
7790,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7791,None,MILD,None,None,None,None,None,None,MILD,None,...,None,None,None,None,None,None,None,None,None,None


In [87]:
symptom_columns = symptom_df.columns

In [217]:
symptom_df['d09fcdbb-9411-46d8-affd-fa5fce2fab00'].value_counts()

NOT_PRESENT    607
PRESENT        170
MILD            41
MODERATE        23
UNSURE           8
SEVERE           2
Name: d09fcdbb-9411-46d8-affd-fa5fce2fab00, dtype: int64

In [199]:
i = 0
for symptom_id in symptom_columns:
    i+=1
    if i in range(130,160):
        print(symptom_df[symptom_id].value_counts())

PRESENT        19
MILD           10
NOT_PRESENT     8
MODERATE        7
Name: 0f9577ab-8209-4f32-91c9-c76349d7a35a, dtype: int64
NOT_PRESENT    9
PRESENT        6
MODERATE       2
Name: 809ba350-88e9-4f94-a442-556393da4970, dtype: int64
PRESENT        3
MILD           2
MODERATE       1
NOT_PRESENT    1
Name: f9fde8c9-7204-4791-93ea-76f14a75c8fb, dtype: int64
PRESENT        5
MILD           4
NOT_PRESENT    2
Name: d11d7068-fce4-449a-a3f0-b227ece2e023, dtype: int64
PRESENT        19
NOT_PRESENT    10
MILD            4
MODERATE        3
SEVERE          2
Name: c0ad1d9a-0859-4e51-9a8f-f5809cb55901, dtype: int64
PRESENT        6
NOT_PRESENT    5
SEVERE         4
MILD           2
MODERATE       1
Name: ec51941e-63d8-4870-b1f4-25f582c0f0b2, dtype: int64
NOT_PRESENT    80
PRESENT         8
MILD            2
Name: 16a590bc-c1a6-4a17-a585-2c49cb30a4dd, dtype: int64
NOT_PRESENT    9
PRESENT        9
MILD           2
SEVERE         1
MODERATE       1
Name: 3cd1a2f4-50f0-45ee-860b-d3be803e68e1, d

Symptom data is a bit weird, we have as severity options: PRESENT, NOT_PRESENT, MILD, MODERATE, SEVERE.

Suggestions: 
None -> 0,
NOT_PRESENT -> 0.1,
MILD -> 0.3,
MODERATE -> 0.5,
PRESENT -> 0.7,
SEVERE -> 1

In [147]:
doctors = []
for x in cases:
    for y in x['outcomes']:
        doctor_id = y['user']['id']
        if doctor_id not in doctors:
            doctors.append(doctor_id)

In [148]:
len(doctors)

73

In [130]:
disease_guesses = []
for x in cases:
    for y in x['outcomes']:
        disease_guess = y['doctor_diseases']
        for z in disease_guess:
            disease_guess_id = z['concept']['id']
            if disease_guess_id not in disease_guesses:
                disease_guesses.append(disease_guess_id)

In [131]:
len(disease_guesses)

455

In [98]:
doctor_ids = []
case_idxs = []
for x in cases:
    for y in x['outcomes']:
        doctor_ids.append(y['user']['id'])
        case_idxs.append(y['card'])

        

In [99]:
doctor_case_df = pd.DataFrame({'doctors' : doctor_ids, 'cards': case_idxs})
doctor_case_df

,doctors,cards
0,e1745fc0-e32c-4dad-a012-8fe1f9b96c5b,427
1,ee213860-a68b-4935-a578-379e81feb191,427
2,957c696f-fef1-4850-aa74-512e13c9c7d3,427
3,5ed83a11-f5fb-40f7-978e-d63e132d18a0,427
4,8ad5b429-7c9b-4f14-8c39-55330ca57f47,427
...,...,...
7972,408f8995-66fd-4efc-bab0-9d78e22fcc96,7791
7973,eb8a4fc9-5248-46cc-b29c-0d6b2b354dbe,7791
7974,eb8a4fc9-5248-46cc-b29c-0d6b2b354dbe,7797
7975,408f8995-66fd-4efc-bab0-9d78e22fcc96,7797


In [105]:
doctor_case_df.groupby('doctors').size().sort_values()


doctors
477e4c87-ce50-40de-9cca-fe30f5666364      3
eb54af71-07fe-4324-b179-0543d79cd6d1      3
ac129e1e-7f24-49d1-b835-7acdc952d9bc      3
d8f827e1-d2c4-494c-a323-29a96c92fa6e      5
41392116-e043-40e0-9440-b2889d3837a1      5
                                       ... 
23343953-40f0-4d35-b457-50431de245a4    339
3c4e5e07-8a05-4f22-b394-2c59bdc5d738    340
a1f62f6c-bc54-4bdc-a71b-ddc0bef4ada8    401
9360d18b-a4f7-424f-bfdf-61abe32eeb62    513
8ad5b429-7c9b-4f14-8c39-55330ca57f47    617
Length: 73, dtype: int64

In [106]:
doctor_case_df.groupby('doctors').size().mean()

109.27397260273973

In [115]:
risk_factors = []
for x in cases:
    for y in x['card']['risk_factors']:
        risk_factor_id = y['concept']['id']
        if risk_factor_id not in risk_factors:
            risk_factors.append(risk_factor_id)
    

In [116]:
len(risk_factors)

189

In [117]:
risk_factor_case_dict = {}

for risk_factor_id in risk_factors:
    risk_factor_case_dict[risk_factor_id] = []


In [118]:
for case in cases:
    case_risk_factors = case['card']['risk_factors']
    case_risk_factor_ids = [x['concept']['id'] for x in case_risk_factors]
    
    for risk_factor_id in risk_factors:
        if risk_factor_id in case_risk_factor_ids:
            idx = case_risk_factor_ids.index(risk_factor_id)
            severity = case_risk_factors[idx]['presence']
            risk_factor_case_dict[risk_factor_id].append(severity)
        else:
            risk_factor_case_dict[risk_factor_id].append(None)

In [119]:
risk_factor_df = pd.DataFrame(risk_factor_case_dict, index = ids)

In [120]:
risk_factor_df

,90699a02-b743-48f4-a99b-40a7252e2255,8129e7ba-440b-448f-8b50-3a52398afe94,74189f81-1a5d-481c-b2e3-7bdf42fd7814,8a12ff56-5f75-4c1a-9f11-5dbabe154c7a,30d9e46e-9bd6-4386-a99a-b078755698a0,5bff05cb-e08c-481e-ab65-69d59a57a682,ea652d05-da0a-495e-9bf8-fc6186dca6da,c4d82b0e-b3b0-4a3f-b6fe-b55ec17c6f4c,8293a444-4718-4b32-9f1f-d6f2803f10ce,5af61a55-631a-449f-8501-eb6d43ceba70,...,cfcc20f5-898b-4c28-9223-60dc03f94a53,4d93c989-95ce-4088-8fcc-2f90263035ea,a24d9e23-596f-4966-9eb2-4336ef9d524e,cdadff6c-1b6b-44ad-9662-31f11930dfce,f6140e7e-18aa-47ed-b25f-a6a6b741d926,7fa2d36e-2d84-4356-94d3-667d21dbf44f,d748bd75-3254-4918-9c3b-8a15cd289162,c3afe8d8-ea1e-45e7-a733-f308dd60162c,401e2dcb-a0ba-4a1a-843b-08c05edb2ef2,fc664706-b201-44a3-92ee-169fc4a19beb
427,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
428,PRESENT,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
429,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
430,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
431,PRESENT,PRESENT,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7772,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7784,PRESENT,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7790,PRESENT,None,NOT_PRESENT,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7791,PRESENT,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [200]:
i = 0
risk_factor_columns = risk_factor_df.columns
for ris in risk_factor_columns:
    i+=1
    if i in range(130,160):
        print(risk_factor_df[ris].value_counts())

PRESENT    2
Name: 1b7ef7f7-a2a3-43bf-9608-8b2c6310494e, dtype: int64
PRESENT    1
Name: 523615a0-65be-4606-a707-4ae21f097392, dtype: int64
PRESENT    2
Name: adf4d0b9-eccb-4c54-a210-2724e8965970, dtype: int64
PRESENT    4
Name: 3affe297-1f1f-4df3-860f-8606141d9629, dtype: int64
NOT_PRESENT    5
PRESENT        3
Name: ecbe5706-2e0e-4c96-b32d-e31b10f6282d, dtype: int64
PRESENT    1
Name: 7acf766d-c079-49e1-8b31-75eaa8bc097e, dtype: int64
NOT_PRESENT    2
PRESENT        1
Name: ad7205dc-e45f-4c45-92de-933721bf59d3, dtype: int64
PRESENT        3
NOT_PRESENT    2
Name: ec7dcef9-d583-4665-8e18-38683d1a51ce, dtype: int64
NOT_PRESENT    3
Name: e1d81bcf-2fc2-40d0-bb5a-168c771808d6, dtype: int64
NOT_PRESENT    10
Name: c63141d3-def0-4994-b042-9e97c0e86adb, dtype: int64
NOT_PRESENT    3
PRESENT        2
Name: 709c13fa-7886-43c3-a5f4-f1e2477dc630, dtype: int64
NOT_PRESENT    2
PRESENT        2
Name: 74cc6754-29ba-4036-860f-e65ea60cff96, dtype: int64
PRESENT        3
NOT_PRESENT    3
Name: d85d08

198 risk factors, only values are PRESENT, NOT_PRESENT, and NONE (1, 0.1 and 0)

In [121]:
triages = []
for x in cases:
    for y in x['outcomes']:
        triage = y['triage']
        if triage not in triages:
            triages.append(triage)

In [123]:
triages

['GP',
 'HOSPITAL',
 'GP_URGENT',
 'PHARMACY',
 'HOSPITAL_URGENT',
 'SELF_CARE',
 '',
 'SPECIALIST_ROUTINE']

Can separate triages into 8 categories of the above, 0 or 1

In [227]:
symptom_dict = {}
for symptom_id in symptoms:
    symptom_dict[symptom_id] = []

    
risk_factor_dict = {}
for risk_factor_id in risk_factors:
    risk_factor_dict[risk_factor_id] = []
    
disease_guess_dict = {}
for disease_guess_id in disease_guesses:
    disease_guess_dict[disease_guess_id] = []
    
triage_dict = {}
for triage in triages:
    triage_dict[triage] = []


In [228]:
def sym_trans(value):
    if value == 'NOT_PRESENT':
        return 0.1
    elif value == 'MILD':
        return 0.2
    elif value == 'UNSURE':
        return 0.35
    elif value == 'MODERATE':
        return 0.5
    elif value == 'PRESENT':
        return 0.7
    elif value == 'SEVERE':
        return 1.0

In [229]:
def risk_trans(value):
    if value == 'NOT_PRESENT':
        return 0.1
    elif value == 'PRESENT':
        return 1.0

In [230]:
doctor_ids = []
for x in cases:
    symptom_ids = [y['concept']['id'] for y in x['card']['symptoms']]
    symptom_severities = [y['severity'] for y in x['card']['symptoms']]
    
    risk_factor_ids = [y['concept']['id'] for y in x['card']['risk_factors']]
    risk_factor_presences = [y['presence'] for y in x['card']['risk_factors']]
    
    for y in x['outcomes']:
        doctor_ids.append(y['user']['id'])
        
        guesses = [z['concept']['id'] for z in y['doctor_diseases']]
        num_guesses = len(guesses)
        
        triage_actual = y['triage']
        
        for sym in symptoms:
            if sym in symptom_ids:
                index = symptom_ids.index(sym)
                symptom_dict[sym].append(sym_trans(symptom_severities[index]))
            else:
                symptom_dict[sym].append(0)
                
        for risk in risk_factors:
            if risk in risk_factor_ids:
                index = risk_factor_ids.index(risk)
                risk_factor_dict[risk].append(risk_trans(risk_factor_presences[index]))
            else:
                risk_factor_dict[risk].append(0)
                
        
        for guess in disease_guesses:
            if guess in guesses:
                disease_guess_dict[guess].append(1/num_guesses)
            else:
                disease_guess_dict[guess].append(0)
        
        for tri in triages:
            if tri == triage_actual:
                triage_dict[tri] = 1
            else:
                triage_dict[tri] = 0
        
        
            

In [231]:
giga_dict = {'doctors': doctor_ids} | symptom_dict | risk_factor_dict | disease_guess_dict | triage_dict

In [232]:
len(giga_dict)

1205

In [233]:
giga_df = pd.DataFrame(giga_dict)

In [234]:
giga_df = giga_df.sort_values('doctors')

In [235]:
giga_df

,doctors,e97c3c8f-fb0d-4817-b6fc-a4a22db6a581,2234f4b8-fb8e-4d11-88f5-6054cb34db12,a40ffca1-8c4d-4a3a-9eaa-8536f4230a43,3948e1d7-a894-4dc9-b293-45be82543289,0e7faf20-6073-49ce-a037-1b7eeb443117,cc0446f9-77b4-41bb-9aa1-c0976a5dcb06,35e2aeb9-932a-40b4-85fa-80c4b3ce21e2,923c9bf2-68d8-4122-8367-8af5f1653a59,172d1f85-8f42-444d-b407-5392fa924d94,...,48d72a85-a215-4333-bfbf-3c269177e518,ed7e4368-929d-4fa8-901f-0424452c1a12,GP,HOSPITAL,GP_URGENT,PHARMACY,HOSPITAL_URGENT,SELF_CARE,,SPECIALIST_ROUTINE
5590,09ec6b50-98cc-4cf1-b072-939bb1a795c5,0.0,0.7,0.0,0.0,0.0,0.0,0.0,0.7,0.0,...,0.0,0.0,1,0,0,0,0,0,0,0
5199,09ec6b50-98cc-4cf1-b072-939bb1a795c5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,0,0,0,0
5927,09ec6b50-98cc-4cf1-b072-939bb1a795c5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,0,0,0,0
5184,09ec6b50-98cc-4cf1-b072-939bb1a795c5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,0,0,0,0
5176,09ec6b50-98cc-4cf1-b072-939bb1a795c5,0.0,0.1,0.0,0.0,0.0,0.0,0.1,0.7,0.7,...,0.0,0.0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5194,f447278d-88a0-4181-aaad-cc4c935666c5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,0,0,0,0
6270,f447278d-88a0-4181-aaad-cc4c935666c5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,0,0,0,0
5539,f447278d-88a0-4181-aaad-cc4c935666c5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,0,0,0,0
5383,f447278d-88a0-4181-aaad-cc4c935666c5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,0,0,0,0


In [236]:
for i in range(len(doctors)):
    giga_df['doctors'] = giga_df.doctors.replace(doctors[i],i)

In [237]:
giga_df

,doctors,e97c3c8f-fb0d-4817-b6fc-a4a22db6a581,2234f4b8-fb8e-4d11-88f5-6054cb34db12,a40ffca1-8c4d-4a3a-9eaa-8536f4230a43,3948e1d7-a894-4dc9-b293-45be82543289,0e7faf20-6073-49ce-a037-1b7eeb443117,cc0446f9-77b4-41bb-9aa1-c0976a5dcb06,35e2aeb9-932a-40b4-85fa-80c4b3ce21e2,923c9bf2-68d8-4122-8367-8af5f1653a59,172d1f85-8f42-444d-b407-5392fa924d94,...,48d72a85-a215-4333-bfbf-3c269177e518,ed7e4368-929d-4fa8-901f-0424452c1a12,GP,HOSPITAL,GP_URGENT,PHARMACY,HOSPITAL_URGENT,SELF_CARE,,SPECIALIST_ROUTINE
5590,46,0.0,0.7,0.0,0.0,0.0,0.0,0.0,0.7,0.0,...,0.0,0.0,1,0,0,0,0,0,0,0
5199,46,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,0,0,0,0
5927,46,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,0,0,0,0
5184,46,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,0,0,0,0
5176,46,0.0,0.1,0.0,0.0,0.0,0.0,0.1,0.7,0.7,...,0.0,0.0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5194,53,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,0,0,0,0
6270,53,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,0,0,0,0
5539,53,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,0,0,0,0
5383,53,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,0,0,0,0


In [238]:
giga_df = giga_df.sort_values('doctors')

In [239]:
giga_df

,doctors,e97c3c8f-fb0d-4817-b6fc-a4a22db6a581,2234f4b8-fb8e-4d11-88f5-6054cb34db12,a40ffca1-8c4d-4a3a-9eaa-8536f4230a43,3948e1d7-a894-4dc9-b293-45be82543289,0e7faf20-6073-49ce-a037-1b7eeb443117,cc0446f9-77b4-41bb-9aa1-c0976a5dcb06,35e2aeb9-932a-40b4-85fa-80c4b3ce21e2,923c9bf2-68d8-4122-8367-8af5f1653a59,172d1f85-8f42-444d-b407-5392fa924d94,...,48d72a85-a215-4333-bfbf-3c269177e518,ed7e4368-929d-4fa8-901f-0424452c1a12,GP,HOSPITAL,GP_URGENT,PHARMACY,HOSPITAL_URGENT,SELF_CARE,,SPECIALIST_ROUTINE
1127,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,0,0,0,0
1165,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,0,0,0,0
1314,0,0.0,0.0,0.0,0.7,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,0,0,0,0
309,0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.5,0.0,...,0.0,0.0,1,0,0,0,0,0,0,0
1319,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7876,72,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,0,0,0,0
7213,72,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,0,0,0,0
6800,72,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,0,0,0,0
6651,72,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7,0.0,...,0.0,0.0,1,0,0,0,0,0,0,0


In [241]:
giga_df.isnull().sum().sum()

368

In [177]:
giga_df.to_csv('Data/processed_data.csv')

In [182]:
giga_df.groupby('doctors').size().argmax()

4

In [183]:
giga_df.groupby('doctors').size().max()

617

In [184]:
df = pd.read_csv('Data/processed_data.csv')

In [185]:
reliable_df = df.loc[df['doctors'] == 4]

In [191]:
reliable_df

,Unnamed: 0,doctors,e97c3c8f-fb0d-4817-b6fc-a4a22db6a581,2234f4b8-fb8e-4d11-88f5-6054cb34db12,a40ffca1-8c4d-4a3a-9eaa-8536f4230a43,3948e1d7-a894-4dc9-b293-45be82543289,0e7faf20-6073-49ce-a037-1b7eeb443117,cc0446f9-77b4-41bb-9aa1-c0976a5dcb06,35e2aeb9-932a-40b4-85fa-80c4b3ce21e2,923c9bf2-68d8-4122-8367-8af5f1653a59,...,48d72a85-a215-4333-bfbf-3c269177e518,ed7e4368-929d-4fa8-901f-0424452c1a12,GP,HOSPITAL,GP_URGENT,PHARMACY,HOSPITAL_URGENT,SELF_CARE,Unnamed: 1204,SPECIALIST_ROUTINE
253,3918,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,0,0,0,0
254,3660,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,0,0,0,0
255,1766,4,0.0,0.1,0.1,0.1,0.1,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,0,0,0,0
256,1726,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,0,0,0,0
257,3755,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7,...,0.0,0.0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,2992,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3,...,0.0,0.0,1,0,0,0,0,0,0,0
866,1413,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,0,0,0,0
867,2159,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,0,0,0,0
868,3118,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,0,0,0,0


In [198]:
reliable_df.isnull().sum().to_string()

'Unnamed: 0                              0\ndoctors                                 0\ne97c3c8f-fb0d-4817-b6fc-a4a22db6a581    0\n2234f4b8-fb8e-4d11-88f5-6054cb34db12    0\na40ffca1-8c4d-4a3a-9eaa-8536f4230a43    0\n3948e1d7-a894-4dc9-b293-45be82543289    0\n0e7faf20-6073-49ce-a037-1b7eeb443117    0\ncc0446f9-77b4-41bb-9aa1-c0976a5dcb06    0\n35e2aeb9-932a-40b4-85fa-80c4b3ce21e2    0\n923c9bf2-68d8-4122-8367-8af5f1653a59    1\n172d1f85-8f42-444d-b407-5392fa924d94    0\nb8d4f595-ac87-4f1d-873a-030b803737b8    0\n524ee59b-5126-4bcc-9d6f-cff17f03048c    0\nef2758f9-d476-43de-855c-a93307cdca4b    0\nfb8c8674-98cb-4233-8148-5b86132cd672    0\n13af929a-06ac-4c7e-9f8d-24cad574c6ef    1\n1d65ae37-c46e-4082-89cd-54e97e6a499b    0\nd09fcdbb-9411-46d8-affd-fa5fce2fab00    1\na97c8252-030c-415f-935b-823afcbb0c54    0\n2ce11d02-308a-4dce-bae6-4104fad1de81    2\n0f0772fd-ea79-43f8-ae47-411b107f1f2c    0\ne00f425e-3e05-4dd3-b0fe-c5632b0f72be    0\nae43d9c2-08cc-41d3-bcff-77194c345b02    0\n07b5c74a-a

In [216]:
reliable_df['d09fcdbb-9411-46d8-affd-fa5fce2fab00'].isnull().sort_values()

253    False
659    False
660    False
661    False
662    False
       ...  
459    False
460    False
461    False
869    False
493     True
Name: d09fcdbb-9411-46d8-affd-fa5fce2fab00, Length: 617, dtype: bool